In [6]:
from models.processing import filter_dataset, load_dataset
from dataclasses import dataclass
from typing import List
import pandas as pd
from gurobipy import *

In [4]:
def load_dataset(data_input):
    """
    Load dataset from the preprocessed CSV
    :return:
    """
    df = pd.read_csv(data_input)
    return df

In [11]:
import random
def generate_distance_matrix(locations):
    """
    Given a list of n tuples representing location coordinates, return an nxn matrix containing all of the distances
    :param locations: list of n tuples representing location coordinates
    :return: nxn matrix containing all of the distances
    """
    dima=[[0 for i in range(len(locations))] for j in range(len(locations))]
    for i in range(len(locations)):
        for j in range(len(locations)):
            if i==j:
                dima[i][j]=0
            elif dima[j][i]!=0:
                dima[i][j]=dima[j][i]
            else:
                dima[i][j]=random.randint(1,101)
    return dima

In [2]:
@dataclass
class Bar:
    name: str
    latitude: str
    longitude: str
    rating: float

In [3]:
@dataclass
class Solution:
    bars = List[Bar]
    total_max_walking_time = float

In [118]:
def get_optimal_route(df, start_time, end_time, bar_num, total_max_walking_time, max_walking_each):
    """

    :param df:
    :param start_time:
    :param end_time:
    :param bar_num:
    :param total_max_walking_time:
    :param max_walking_each:
    :return: A Solution class object representing the optimal solution
    """
    
    #parameters
    bigm=999999
    m=Model("opt_route")
    locations=df['name']
    ratings=df['stars']
    open_times=df['open']
    close_times=df['close']
    y=[]
    x=[[0 for j in range(len(locations))] for i in range(len(locations))]
    z=[[[0 for j in range(len(locations))] for i in range(len(locations))] for k in range(bar_num-1)]
    dima=generate_distance_matrix(locations)
    waittimes=wait_time(df)
    
    #create decision variables
    for loc in locations:
        y.append(m.addVar(vtype=GRB.BINARY, name="y_{}".format(loc)))
    
    for i in range(len(locations)):
        for j in range(len(locations)):
            x[i][j]=m.addVar(vtype=GRB.BINARY,name="x_{},{}".format(locations[i],locations[j]))
            
    for k in range(bar_num-1):
        for i in range(len(locations)):
            for j in range(len(locations)):
                z[k][i][j]=m.addVar(vtype=GRB.BINARY,name="z_{},{},{}".format(k,locations[i],locations[j]))
     
    #objective function
    m.setObjective(quicksum([y[i]*ratings[i] for i in range(len(locations))]),GRB.MAXIMIZE)
    
    #constraints
        #no movements between the same bar
#     for i in range(len(locations)):
#         m.addConstr(x[i][i]==0)
    
        #inbound arcs
#     for j in range(len(locations)):
#         m.addConstr(quicksum([x[i][j] for i in range(len(locations))])==y[j])
        
        #outbound arcs
#     for i in range(len(locations)):
#         m.addConstr(quicksum([x[i][j] for j in range(len(locations))])==y[i])
    
        #subtours
    
        #Locations visited
    m.addConstr(quicksum(y)==bar_num)
    
        #max total walk time
#     m.addConstr(quicksum([x[i][j]*dima[i][j] for i in range(len(locations)) for j in range (len(locations))])<=total_max_walking_time)  
    
#         #max walk time between locations
#     for i in range(len(locations)):
#         for j in range(len(locations)):
#             m.addConstr(x[i][j]*dima[i][j]<=max_walking_each)
    
        #rules about z
        #no movements between the same bar
    for k in range(bar_num-1):
        for i in range(len(locations)):
            m.addConstr(z[k][i][i]==0)
            
        #froms/tos upper bound
    for i in range(len(locations)):
        m.addConstr(quicksum([z[k][i][j] for k in range(bar_num - 1) for j in range(len(locations))])
                    +quicksum([z[k][j][i] for k in range(bar_num - 1) for j in range (len(locations))])
                    <= bigm*y[i])
        
        #froms/tos lower bound
    for i in range(len(locations)):
        m.addConstr(quicksum([z[k][i][j] for k in range(bar_num - 1) for j in range(len(locations))])
                    +quicksum([z[k][j][i] for k in range(bar_num - 1) for j in range (len(locations))])
                    >= y[i]/bigm)
        
        #can only have one 1 per movement matrix
    for k in range(bar_num-1):
        m.addConstr(quicksum([z[k][i][j] for i in range(len(locations)) for j in range (len(locations))])==1)
    
# #     #must have the same value as xij
#     for i in range(len(locations)):
#         for j in range(len(locations)):
#             m.addConstr(quicksum([z[k][i][j] for k in range(bar_num-1)])==x[i][j])
              
    #make sure we don't vist the same bar twice
    #dimension1
    for i in range(len(locations)):
        m.addConstr(quicksum([z[k][i][j] for j in range(len(locations)) for k in range (bar_num-1)])<=1)
     
     #dimension2
    for j in range(len(locations)):
        m.addConstr(quicksum([z[k][i][j] for i in range(len(locations)) for k in range (bar_num-1)])<=1)
        
     #have to start from the bar you previously went to
    for k in range(1,bar_num-1):
        for i in range(len(locations)):
            m.addConstr(quicksum([z[k-1][j][i] for j in range(len(locations))])==quicksum([z[k][i][j] for j in range(len(locations))]))
    
    #open  time - only distance is considered for the time being
    for zed in range(bar_num):
        m.addConstr(start_time+quicksum([z[k][i][j]*dima[i][j] for k in range(zed-1)])
                    >=quicksum([open_times[i]*quicksum(z[zed][i]) for i in range(len(locations))])) 
       
    #close time - only distance is considered for the time beeing
    for zed in range(bar_num):
        m.addConstr(start_time+quicksum([z[k][i][j]*dima[i][j] for k in range(zed-1)])
                    <=quicksum([close_times[i]*quicksum(z[zed][i]) for i in range(len(locations))])) 
     
    #m.setParam('TimeLimit', 30)
    #m.setParam('MIPFocus',1)
    m.optimize()
    return m,z,y,x,dima

In [5]:
def get_pareto_routes(df, start_time, end_time, bar_num, total_max_walking_time, max_walking_each):
    """
    Returns total_max_walking_time / 5 suggested routes (e.g. one for 0-5 mins walk, one for 5-10 mins walk, etc.).
    Pareto routes contain
    :param df:
    :param start_time:
    :param end_time:
    :param bar_num:
    :param total_max_walking_time:
    :param max_walking_each:
    :return: A list of Solutions
    """
    solutions = []
    for max_time in range(0, int(total_max_walking_time), 5):
        solutions.append(get_optimal_route(df, start_time, end_time, bar_num, max_time, max_walking_each))
    pass

In [6]:
def crawl_model(date, start_time, end_time, budget, bar_num,  total_max_walking_time,  max_walking_each, min_review_ct,
                min_review, city):
    """
    :param date:
    :param start_time:
    :param end_time:
    :param budget:
    :param bar_num:
    :param total_max_walking_time:
    :param max_walking_each:
    :param min_review_ct:
    :param min_review:
    :param city:
    :return:
    """

    df = load_dataset()
    df = filter_dataset(df, min_review_ct, min_review, date, budget, city)
    pareto_df = get_pareto_routes(df, start_time, end_time, bar_num, total_max_walking_time, max_walking_each)

    return pareto_df

In [111]:
toronto_data=load_dataset('data\\business_example_HBmod.csv')

In [115]:
pd.DataFrame(dima).to_csv('dima.csv')

In [28]:
strs=toronto_data['stars']
strs[187]

3.0

In [119]:
wait_time=generate_distance_matrix
optrout,z,y,x,dima=get_optimal_route(toronto_data[:50], 12, 17, 6, 999999, 999999)

IndexError: list index out of range

In [113]:
for k in range(6-1):
    for i in range(50):
        for j in range(50):
            if z[k][i][j].x!=0:
                print(z[k][i][j])

<gurobi.Var z_0,Koerner Hall,Stirling Room (value 1.0)>
<gurobi.Var z_1,Stirling Room,Another Bar (value 1.0)>
<gurobi.Var z_2,Another Bar,Koricancha Restaurant (value 1.0)>
<gurobi.Var z_3,Koricancha Restaurant,Bathurst Local (value 1.0)>
<gurobi.Var z_4,Bathurst Local,I'll Be Seeing You (value 1.0)>


In [117]:
toronto_data['open']

0      17.0
1      11.0
2      18.0
3      16.0
4      22.0
5      13.0
6      20.0
7       9.0
8      17.5
9      10.0
10     11.0
11     18.0
12     12.0
13     17.0
14     11.0
15      8.0
16     17.0
17     16.0
18     10.0
19     11.5
20     11.0
21     16.0
22     18.0
23     16.0
24     11.0
25     15.0
26     20.0
27     16.0
28     11.5
29     19.0
       ... 
958    11.0
959    21.0
960    11.0
961    16.0
962    11.0
963    18.0
964    17.0
965    17.0
966    17.0
967    10.5
968    11.0
969    10.5
970    15.0
971    11.0
972    11.0
973    17.0
974    10.0
975    10.0
976    11.5
977    13.0
978    11.0
979    20.0
980    11.0
981    17.0
982    11.0
983     9.0
984    11.5
985    10.0
986    17.0
987    13.0
Name: open, Length: 988, dtype: float64